### Background Information

Moni is a financial technology company that provides various financial services in diverse Africa countries which includes Nigeria, Benin Republic, Guinea and so on. focus. The company was founded with the goal of providing simple and innovative access to finance to the African community.

Moni offers services such as high-yielding savings plans, business loans, insurance, and cash insurance policies. The company's savings plan allows users to create multiple savings plans to fund their lifestyle and dreams while earning high interest rates. Their business loans are designed to help small and medium-sized enterprises (SMEs) access flexible loans at competitive interest rates. They also offer insurance policies that protect businesses in the event of cash loss. 

Central to savings, Moni enables over 20k young people across Africa to save towards a goal.  This project is geared towards analyzing and visualizing patterns and insight from this sample data from Moni.

### Loading Libaries

In [1]:
# Data wragling libraries
import pandas as pd
import numpy as np

# Ignoring warning
import warnings
warnings.filterwarnings('ignore')

# Creating, storing and connecting to a database
import mysql.connector
from mysql.connector import Error

### Loading Dataset

In [2]:
df = pd.read_csv("table_data.csv")

### Connecting to MySQL

In [3]:
mysql_connect = mysql.connector.connect(host='localhost', user='root', password ='Adizoriola+1')
print(mysql_connect)

### Creating Database

In [4]:
def CreateDatabase (mysql_connection, db_name) :
    cursor = mysql_connection.cursor()
    query = f"CREATE DATABASE {db_name}"
    try:
        cursor.execute(query)
        print ('Database created successfully')
    except Error as err:
        print(f'Error : {err}')

In [6]:
CreateDatabase(mysql_connect, 'MoniSavingss')

Database created successfully


In [7]:
def DatabaseConnector(user_name, host_name, password, database):
    connection = None
    try:
        connection = mysql.connector.connect (host = host_name,user = user_name,
                                              passwd = password, database = database)
        print ('Mysql Database connected successfully')
    except Error as err:
        
        print (f'Error: {err}\n')
    return connection

In [8]:
db_connection = DatabaseConnector(user_name = 'root', host_name = 'localhost',
                                password = 'Adizoriola+1', database = 'MoniSavingss')

Mysql Database connected successfully


In [9]:
# Creating the function to execute query

def ExecuteQuery(db_connection, query):
    cursor = db_connection.cursor()
    result = None
    try:
        cursor.execute (query)
        result = cursor.fetchall()
    except Error as err:
        print(f'Error : {err}')
    return result

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Funding _ Primary _ ID                500 non-null    object 
 1   Funding _ Primary _ Type              500 non-null    object 
 2   ID                                    500 non-null    object 
 3   Schedule _ Format                     424 non-null    object 
 4   Schedule _ Free Withdrawals Left      500 non-null    int64  
 5   Schedule _ Frequency                  424 non-null    object 
 6   Schedule _ History _ Last Date        500 non-null    object 
 7   Schedule _ History _ Next Date        500 non-null    object 
 8   Schedule _ Interest Per Day           500 non-null    float64
 9   Schedule _ Interest Withdrawals Type  76 non-null     object 
 10  Schedule _ Settings _ Day             424 non-null    float64
 11  Schedule _ Settings

In [5]:
### Putting the date features in a list in order to change the datatype

date_features= [x for x in df.columns if "Date" in x or "Created At" in x or "Modified At"in x ]
date_features

['Schedule _ History _ Last Date',
 'Schedule _ History _ Next Date',
 'Start Date',
 'End Date',
 'Next Debit Date',
 'Created At',
 'Modified At']

In [6]:
## Converting the date features to a datetime feature
for col in date_features:
    df[col] = pd.to_datetime(df[col])

In [7]:
# checking the percentage of missing values

def missing_values_tables(df):
    # Total Missing Values
    missing_values = df.isnull().sum()
    
    # Percentage of Missing Values
    missing_value_percent = 100 * (df.isnull().sum()/len(df))
    
    # Make a table with the results
    missing_value_table = pd.concat([missing_values, missing_value_percent], axis = 1)
    
    # Rename the columns 
    missing_values_table_rename_columns = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
                        
    # Sort the table by percentage of missing descending
    missing_values_table_rename_columns = missing_values_table_rename_columns[missing_values_table_rename_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    
    # Printing some sumarry information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(missing_values_table_rename_columns.shape[0]) +
              " columns that have missing values.")
                                   
     # Return the dataframe with missing information
    return missing_values_table_rename_columns

In [8]:
missing_values_tables(df)

Your selected dataframe has 26 columns.
There are 6 columns that have missing values.


,Missing Values,% of Total Values
Schedule _ Interest Withdrawals Type,424,84.8
Schedule _ Settings _ Time,202,40.4
Schedule _ Format,76,15.2
Schedule _ Frequency,76,15.2
Schedule _ Settings _ Day,76,15.2
Next Debit Date,76,15.2


In [9]:
# Drop Columns that might not be useful for analysis

df = df.drop(columns=['Schedule _ History _ Last Date', 'Schedule _ Interest Withdrawals Type', 'Schedule _ History _ Next Date', 'Schedule _ Settings _ Day',
                         'Schedule _ Settings _ Time', 'Next Debit Date'])

In [24]:
create_table = """
CREATE TABLE dataset(
  funding_primary_id VARCHAR(255) NULL,
  funding_primary_type VARCHAR(255) NULL,
  ID VARCHAR(255) NULL,
  schedule_format VARCHAR(255) NULL,
  schedule_free_withdrawals_left INT NULL,
  schedule_frequency VARCHAR(255) NULL,
  schedule_interest_per_day FLOAT NULL,
  savings_type_id INT NULL,
  status_id INT NULL,
  owner_id VARCHAR(255) NULL,
  principal_amount FLOAT NULL,
  target_amount FLOAT NULL,
  amount_per_frequency FLOAT NULL,
  interest FLOAT NULL,
  title VARCHAR(255) NULL,
  duration INT NULL,
  start_date DATETIME NULL,
  end_date DATETIME NULL,
  created_at DATETIME NULL,
  modified_at DATETIME NULL
);
"""
ExecuteQuery(db_connection, create_table)


[]

In [26]:
query = 'DESCRIBE dataset'
ExecuteQuery(db_connection, query)

[('funding_primary_id', 'varchar(255)', 'YES', '', None, ''),
 ('funding_primary_type', 'varchar(255)', 'YES', '', None, ''),
 ('ID', 'varchar(255)', 'YES', '', None, ''),
 ('schedule_format', 'varchar(255)', 'YES', '', None, ''),
 ('schedule_free_withdrawals_left', 'int', 'YES', '', None, ''),
 ('schedule_frequency', 'varchar(255)', 'YES', '', None, ''),
 ('schedule_interest_per_day', 'float', 'YES', '', None, ''),
 ('savings_type_id', 'int', 'YES', '', None, ''),
 ('status_id', 'int', 'YES', '', None, ''),
 ('owner_id', 'varchar(255)', 'YES', '', None, ''),
 ('principal_amount', 'float', 'YES', '', None, ''),
 ('target_amount', 'float', 'YES', '', None, ''),
 ('amount_per_frequency', 'float', 'YES', '', None, ''),
 ('interest', 'float', 'YES', '', None, ''),
 ('title', 'varchar(255)', 'YES', '', None, ''),
 ('duration', 'int', 'YES', '', None, ''),
 ('start_date', 'datetime', 'YES', '', None, ''),
 ('end_date', 'datetime', 'YES', '', None, ''),
 ('created_at', 'datetime', 'YES', '', 

In [33]:
# Convert DataFrame to tuple for ease of inserting it into the table created

tuple_of_tuples = tuple(df.to_records(index=False))


### Data cleaning with Python

In [12]:
df.head()

,Funding _ Primary _ ID,Funding _ Primary _ Type,ID,Schedule _ Format,Schedule _ Free Withdrawals Left,Schedule _ Frequency,Schedule _ Interest Per Day,Savings Type ID,Status ID,Owner ID,Principal Amount,Target Amount,Amount Per Frequency,Interest,Title,Duration,Start Date,End Date,Created At,Modified At
0,"""95510a7b-6103-4e95-bb85-eb105ba3b1b0""","""wallet""",000edeb5-8513-4194-83dc-90b505f050ed,"""regular""",1,"""monthly""",0.0520,2,3,f54ea357-ed1c-4399-99e3-69ccd25934d7,3000000,36000000,3000000,19.0,Save 30k Monthly,12,2023-01-04 16:18:00,2024-01-04 16:18:00,2023-01-04 16:18:00,2023-02-10 01:59:00
1,"""ba6fb253-faa7-4fc3-963b-e7a45149495a""","""wallet""",00b88550-09e1-41ab-8a62-b74c120b5470,"""regular""",1,"""weekly""",0.0219,2,3,427b2439-7112-4137-be9d-00a42f5a7dda,500000,6500000,500000,8.0,Ashedzi,3,2022-06-26 10:01:00,2022-09-26 10:01:00,2022-06-26 10:01:00,2022-11-03 01:36:00
2,"""8559244b-64b5-4040-8ae2-b99fcf38cf8b""","""wallet""",00c57cd9-d016-4309-a540-f27b9ed6d999,"""regular""",2,"""daily""",0.0260,2,26,41ab9b38-b240-41e5-a123-e1e91556f46f,10000,1530000,10000,9.5,davings,5,2022-08-07 04:59:00,2023-01-07 04:59:00,2022-08-07 04:59:00,2023-01-07 05:00:00
3,"""f9a6b320-a46b-4c72-b949-5c9f2f350e45""","""wallet""",015a1980-0da9-4844-9dc8-f321f5a28780,"""target""",2,"""monthly""",0.0260,2,1,b0cb0cb1-a6d0-439b-8ae7-e04217a9021a,2000000,10000000,2000000,9.5,Big Chief __,5,2022-10-15 23:47:00,2023-03-15 23:47:00,2022-10-15 23:47:00,2023-02-07 00:03:00
4,"""56042136-c2ab-4c08-a507-aabed521b09b""","""wallet""",01991de1-e1be-4eac-b32a-b272d24add17,"""regular""",0,"""daily""",0.0190,2,3,24fbd5f2-357d-4eb3-862e-892ec5c1dd3f,1000000,31000000,1000000,7.0,new,1,2023-01-26 03:52:00,2023-02-26 03:52:00,2023-01-26 03:52:00,2023-01-26 04:01:00


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Funding _ Primary _ ID            500 non-null    object        
 1   Funding _ Primary _ Type          500 non-null    object        
 2   ID                                500 non-null    object        
 3   Schedule _ Format                 424 non-null    object        
 4   Schedule _ Free Withdrawals Left  500 non-null    int64         
 5   Schedule _ Frequency              424 non-null    object        
 6   Schedule _ Interest Per Day       500 non-null    float64       
 7   Savings Type ID                   500 non-null    int64         
 8   Status ID                         500 non-null    int64         
 9   Owner ID                          500 non-null    object        
 10  Principal Amount                  500 non-null    

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Schedule _ Free Withdrawals Left,500.0,1.032000e+00,9.168737e-01,0.000,0.00,1.000,2.000000e+00,2.000000e+00
Schedule _ Interest Per Day,500.0,3.592580e-02,1.976299e-02,0.019,0.02,0.026,5.200000e-02,1.530000e-01
Savings Type ID,500.0,2.128000e+00,3.686296e-01,1.000,2.00,2.000,2.000000e+00,3.000000e+00
Status ID,500.0,4.744000e+00,8.056070e+00,1.000,1.00,1.000,3.000000e+00,2.600000e+01
Principal Amount,500.0,1.233855e+07,5.477724e+07,10000.000,100000.00,500000.000,2.589286e+06,5.000000e+08
Target Amount,500.0,5.875721e+07,2.418265e+08,90000.000,2600000.00,9947500.000,3.000000e+07,2.147484e+09
Amount Per Frequency,500.0,2.478747e+06,1.090338e+07,0.000,32923.50,200000.000,1.000000e+06,1.000000e+08
Interest,500.0,1.295700e+01,6.677037e+00,7.000,7.50,9.500,1.900000e+01,2.800000e+01
Duration,500.0,5.376000e+00,4.076255e+00,1.000,2.00,4.000,9.000000e+00,1.200000e+01


In [16]:
df = df.fillna('NULL')

In [17]:
# Checking the unique variable in the object features
print(df.groupby('Funding _ Primary _ Type').size())
print('\n')
print(df.groupby('Schedule _ Format').size())
print('\n')
print(df.groupby('Schedule _ Free Withdrawals Left').size())
print('\n')
print(df.groupby('Savings Type ID').size())
print('\n')
print(df.groupby('Schedule _ Frequency').size())
print('\n')

Funding _ Primary _ Type
"card"       89
"wallet"    411
dtype: int64


Schedule _ Format
"regular"    334
"target"      90
NULL          76
dtype: int64


Schedule _ Free Withdrawals Left
0    202
1     80
2    218
dtype: int64


Savings Type ID
1      6
2    424
3     70
dtype: int64


Schedule _ Frequency
"bi-weekly"     12
"daily"        189
"monthly"      126
"weekly"        97
NULL            76
dtype: int64




In [18]:
# Encoding the Schedule format and Scheduled frew properly.
default_map = {'"bi-weekly"' : 'bi-weekly', '"daily"': 'daily',  '"monthly"': 'monthly', 
               '"weekly"': 'weekly', 'NULL': 'Not specified'}
df['Schedule _ Frequency'] = df['Schedule _ Frequency'].map(default_map)


def_map = {'"regular"' : "regular", '"target"': "target", 'NULL': 'Not specified'}
df['Schedule _ Format'] = df['Schedule _ Format'].map(def_map)


In [22]:
def_map = {'"card"' : "card", '"wallet"': "wallet"}
df['Funding _ Primary _ Type'] = df['Funding _ Primary _ Type'].map(def_map)

In [23]:
print(df.groupby('Schedule _ Frequency').size())
print('\n')
print(df.groupby('Schedule _ Format').size())
print('\n')
print(df.groupby('Funding _ Primary _ Type').size())

Schedule _ Frequency
Not specified     76
bi-weekly         12
daily            189
monthly          126
weekly            97
dtype: int64


Schedule _ Format
Not specified     76
regular          334
target            90
dtype: int64


Funding _ Primary _ Type
card       89
wallet    411
dtype: int64


In [24]:
# Encoding Numerical Variable Duration

df.groupby('Duration').size()

Duration
1     120
2       9
3     120
4      16
5      30
6      54
7      19
8       5
9       4
10     14
11      5
12    104
dtype: int64

In [25]:
# mapping
def_map = {1 : "1 MONTH", 2 : "2 MONTHS", 3 : "3 MONTHS", 4 : "4 MONTHS", 5 : "5 MONTHS",
          6 : "6 MONTHS", 7 : "7 MONTHS", 8 : "8 MONTHS", 9 : "9 MONTHS", 10 : "10 MONTHS", 
          11 : "11 MONTHS", 12 : "12 MONTHS"}
df['Duration'] = df['Duration'].map(def_map)

In [26]:
df.groupby('Duration').size()

Duration
1 MONTH      120
10 MONTHS     14
11 MONTHS      5
12 MONTHS    104
2 MONTHS       9
3 MONTHS     120
4 MONTHS      16
5 MONTHS      30
6 MONTHS      54
7 MONTHS      19
8 MONTHS       5
9 MONTHS       4
dtype: int64

In [29]:
df.groupby('Savings Type ID').size()

Savings Type ID
1      6
2    424
3     70
dtype: int64

In [30]:
def_map = {1 : "Category 1", 2: "Category 2", 3: "Category 3"}
df['Savings Type ID'] = df['Savings Type ID'].map(def_map)

df.groupby('Savings Type ID').size()

Savings Type ID
Category 1      6
Category 2    424
Category 3     70
dtype: int64

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Funding _ Primary _ ID            500 non-null    object        
 1   Funding _ Primary _ Type          500 non-null    object        
 2   ID                                500 non-null    object        
 3   Schedule _ Format                 500 non-null    object        
 4   Schedule _ Free Withdrawals Left  500 non-null    int64         
 5   Schedule _ Frequency              500 non-null    object        
 6   Schedule _ Interest Per Day       500 non-null    float64       
 7   Savings Type ID                   500 non-null    object        
 8   Status ID                         500 non-null    int64         
 9   Owner ID                          500 non-null    object        
 10  Principal Amount                  500 non-null    

In [32]:
df.to_csv('table_csv1')